### Curso de Python do Maurão
#### Aula 2 - Analisando uma base de dados "real" 


#####  **Objetivos:** 

 - Carga de uma base de dados coletados a partir de eventos reais
 - Fazer uma análise exploratória 
 - Análise de correlação
 - Elaboração de mapa temático

1) Imports e configuração do ambiente

In [ ]:
import pandas as pd
from datetime import datetime
import seaborn as sns
sns.set() # notebook
pd.options.mode.chained_assignment = None

2) Carregando os dados

In [ ]:
start = datetime.now()
df = pd.read_csv(r'D:\Atrium\docs\CursoPython\data\yellow_tripdata_2019-12.csv')
print(datetime.now() - start)

3) Primeiras explorações

In [ ]:
print(df.shape)
print(df.columns)
print(df.dtypes)
print(df.describe())
sp = df.shape

4) Configurando a formatação dos números

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

5) Exibindo alguns dados

In [ ]:
df.head()

In [ ]:
df.tail()

6) Lidando com os valores não informados

In [ ]:
df.columns[df.isna().any()]

In [ ]:
df = df.dropna()

In [ ]:
sp[0] - df.shape[0]

7) Primeiras análises

In [ ]:
sns.distplot(df['passenger_count'])

In [ ]:
sns.distplot(df['passenger_count'],kde=False)

8) Consertando o tipo de dado de campos de data/hora

In [ ]:
df['tpep_pickup_datetime']=pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime']=pd.to_datetime(df['tpep_dropoff_datetime'])

In [ ]:
df.dtypes

9) Criando as colunas de dias da semana

In [ ]:
df['dropoff_day']=df['tpep_dropoff_datetime'].dt.day_name()
df['pickup_day']=df['tpep_pickup_datetime'].dt.day_name()

10) Mais algumas análises exploratórias

In [ ]:
ax=sns.countplot(x='dropoff_day',data=df)

In [ ]:
ax=sns.countplot(df['dropoff_day'])

In [ ]:
ax=sns.countplot(x='dropoff_day',data=df)
ax.set_title('Corridas por dia da semana')

11) Criando campos com as horas de pickup e dropoff

In [ ]:
df['pickup_hour']=df['tpep_pickup_datetime'].dt.hour
df['dropoff_hour']=df['tpep_dropoff_datetime'].dt.hour

In [ ]:
ax=sns.countplot(x='pickup_hour',data=df)

In [ ]:
df['trip_duration']=df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']

In [ ]:
import numpy as np
df['trip_duration']=(df['tpep_dropoff_datetime']-df['tpep_pickup_datetime'])/np.timedelta64(1,'m')

12) Investigando outliers

In [ ]:
print(df['trip_duration'].max())
print(df['trip_duration'].min())

In [ ]:
df[df['trip_duration'] < 0]

In [ ]:
df=df[df['trip_duration'] >= 0]

In [ ]:
sns.boxplot(x=df['trip_duration'])

In [ ]:
Q1 = df['trip_duration'].quantile(0.25)
Q3 = df['trip_duration'].quantile(0.75)
print(Q1,Q3)

In [ ]:
df[(df['trip_duration'] < Q1) | (df['trip_duration'] > Q3)].count()

In [ ]:
df=df[(df['trip_duration'] >= Q1) & (df['trip_duration'] <= Q3)]

In [ ]:
df['trip_duration'].max()

13) Outros gráficos

In [ ]:
sns.boxplot(x=df['trip_duration'])

In [ ]:
sns.violinplot(x=df['trip_duration'],orient='vert')

In [ ]:
import matplotlib.pyplot as plt
figure,ax=plt.subplots(nrows=1,ncols=2,figsize=(10,5))
df.pickup_hour.hist(bins=24,ax=ax[0])
ax[0].set_title('Distribuição início da corrida')
df.dropoff_hour.hist(bins=24,ax=ax[1])

14) Análise correlação

In [ ]:
start = datetime.now()
sns.catplot(x="pickup_day",y="trip_duration",kind="bar",data=df,height=6,aspect=1)
plt.title('Média de duração da corrida por dia da semana')
print(datetime.now() - start)

15) Mapas!

In [ ]:
import geopandas as gpd
nyc = gpd.read_file(r'D:\Atrium\docs\CursoPython\data\nyc_taxi_zones.geojson')
nyc.head()

In [ ]:
txzones=df.groupby(['pulocationid']).count()

In [ ]:
map = pd.merge(nyc,txzones,how='left',left_on=['LocationID'], right_on=['pulocationid'])

In [ ]:
map.plot()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20, 20))
map.plot(column='vendorid',cmap='Greens',ax=ax)
ax.set_title('Corridas iniciadas por zona',fontsize=32)
ax.axis('off')
vmin=map['vendorid'].min()
vmax=map['vendorid'].max()
sm = plt.cm.ScalarMappable(cmap='Greens', norm=plt.Normalize(vmin=vmin, vmax=vmax))
cbar = fig.colorbar(sm)